In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score,KFold
from imblearn.over_sampling import RandomOverSampler

In [3]:
df = pd.read_csv("BRCA Data.csv")
df.head(3)

,Patient_ID,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
1,TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
2,TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive


In [4]:
df.shape

(341, 16)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Patient_ID          334 non-null    object 
 1   Age                 334 non-null    float64
 2   Gender              334 non-null    object 
 3   Protein1            334 non-null    float64
 4   Protein2            334 non-null    float64
 5   Protein3            334 non-null    float64
 6   Protein4            334 non-null    float64
 7   Tumour_Stage        334 non-null    object 
 8   Histology           334 non-null    object 
 9   ER status           334 non-null    object 
 10  PR status           334 non-null    object 
 11  HER2 status         334 non-null    object 
 12  Surgery_type        334 non-null    object 
 13  Date_of_Surgery     334 non-null    object 
 14  Date_of_Last_Visit  317 non-null    object 
 15  Patient_Status      321 non-null    object 
dtypes: float

In [6]:
df.isnull().sum()

Patient_ID             7
Age                    7
Gender                 7
Protein1               7
Protein2               7
Protein3               7
Protein4               7
Tumour_Stage           7
Histology              7
ER status              7
PR status              7
HER2 status            7
Surgery_type           7
Date_of_Surgery        7
Date_of_Last_Visit    24
Patient_Status        20
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().sum()

Patient_ID            0
Age                   0
Gender                0
Protein1              0
Protein2              0
Protein3              0
Protein4              0
Tumour_Stage          0
Histology             0
ER status             0
PR status             0
HER2 status           0
Surgery_type          0
Date_of_Surgery       0
Date_of_Last_Visit    0
Patient_Status        0
dtype: int64

In [9]:
# for i in df.iloc[:,2:]:
#     status = df[i].value_counts()
#     fig = px.pie(values= status.values, names= status.index, hole= 0.5,title= i, width= 700,height= 400)
#     fig.show()
    

In [10]:
df.columns

Index(['Patient_ID', 'Age', 'Gender', 'Protein1', 'Protein2', 'Protein3',
       'Protein4', 'Tumour_Stage', 'Histology', 'ER status', 'PR status',
       'HER2 status', 'Surgery_type', 'Date_of_Surgery', 'Date_of_Last_Visit',
       'Patient_Status'],
      dtype='object')

In [11]:
df.drop(['Patient_ID','ER status', 'PR status', 'Date_of_Surgery', 'Date_of_Last_Visit'],axis=1,inplace= True)
df.head(3)

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,HER2 status,Surgery_type,Patient_Status
0,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Negative,Modified Radical Mastectomy,Alive
1,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Negative,Lumpectomy,Dead
2,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Negative,Other,Alive


In [12]:
encode = pd.get_dummies(df[['Gender','Tumour_Stage','Histology','HER2 status','Surgery_type','Patient_Status']],drop_first=True).astype(int)
encode.head(3)

,Gender_MALE,Tumour_Stage_II,Tumour_Stage_III,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,HER2 status_Positive,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy,Patient_Status_Dead
0,0,0,1,0,0,0,1,0,0,0
1,0,1,0,0,1,0,0,0,0,1
2,0,0,1,0,0,0,0,1,0,0


In [13]:
data = pd.concat([df,encode],axis=1)
data.head(3)

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,HER2 status,Surgery_type,...,Gender_MALE,Tumour_Stage_II,Tumour_Stage_III,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,HER2 status_Positive,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy,Patient_Status_Dead
0,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Negative,Modified Radical Mastectomy,...,0,0,1,0,0,0,1,0,0,0
1,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Negative,Lumpectomy,...,0,1,0,0,1,0,0,0,0,1
2,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Negative,Other,...,0,0,1,0,0,0,0,1,0,0


In [14]:
data.select_dtypes(include='object').columns

Index(['Gender', 'Tumour_Stage', 'Histology', 'HER2 status', 'Surgery_type',
       'Patient_Status'],
      dtype='object')

In [15]:
data.drop(data.select_dtypes(include='object').columns,axis=1,inplace=True)

In [16]:
data.head()

,Age,Protein1,Protein2,Protein3,Protein4,Gender_MALE,Tumour_Stage_II,Tumour_Stage_III,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,HER2 status_Positive,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy,Patient_Status_Dead
0,36.0,0.080353,0.42638,0.54715,0.273680,0,0,1,0,0,0,1,0,0,0
1,43.0,-0.420320,0.57807,0.61447,-0.031505,0,1,0,0,1,0,0,0,0,1
2,69.0,0.213980,1.31140,-0.32747,-0.234260,0,0,1,0,0,0,0,1,0,0
3,56.0,0.345090,-0.21147,-0.19304,0.124270,0,1,0,0,0,0,1,0,0,0
4,56.0,0.221550,1.90680,0.52045,-0.311990,0,1,0,0,0,0,0,1,0,1


In [17]:
data.shape

(317, 15)

In [18]:
ran = RandomOverSampler()
X,Y = ran.fit_resample(data.iloc[:,:-1],data['Patient_Status_Dead'])

In [20]:
X.shape

(510, 14)

In [21]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X[['Age','Protein1','Protein2','Protein3','Protein4']])
X_trans = pd.DataFrame(std.fit_transform(X[['Age','Protein1','Protein2','Protein3','Protein4']]),columns= ['Age','Protein1','Protein2','Protein3','Protein4'])


In [22]:
X_trans

,Age,Protein1,Protein2,Protein3,Protein4
0,-1.694410,0.232930,-0.679807,1.092514,0.450721
1,-1.156478,-0.739037,-0.507129,1.207789,-0.034098
2,0.841555,0.492342,0.327663,-0.405130,-0.356196
3,-0.157462,0.746869,-1.405909,-0.174941,0.213368
4,-0.157462,0.507038,1.005442,1.046795,-0.479678
...,...,...,...,...,...
505,0.764707,-0.427805,-0.185691,0.287000,0.312544
506,-0.311157,0.076939,0.405982,-0.697187,-0.789981
507,-0.849089,-0.292029,1.086379,0.142569,0.072069
508,0.611012,0.632894,0.428294,0.701553,1.344106


In [23]:
xx = X.iloc[:,5:]

In [24]:
xx.shape

(510, 9)

In [25]:
X_trans.shape

(510, 5)

In [26]:
X_final = pd.concat([X_trans.reset_index(drop=True),
                     xx.reset_index(drop=True)], axis=1)


In [27]:
X_final.shape

(510, 14)

In [28]:
Y.value_counts()

Patient_Status_Dead
0    255
1    255
Name: count, dtype: int64

In [29]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_final,Y,test_size=0.2,random_state=42)

In [ ]:
# def model(ml):
#     Model = ml
#     Model.fit(x_train,y_train)
#     Model.score(x_test,y_test)
#     print(Model.score(x_test,y_test))

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
dfir = { 'criterion': ["gini", "entropy", "log_loss"],
    'splitter':['best', 'random'],
    'max_depth':[i for i in range(2,20)]}

In [33]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
gd =GridSearchCV(DecisionTreeClassifier(),param_grid=dfir,cv=5)
gd.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15, 16, 17, 18, 19],
                         'splitter': ['best', 'random']})

In [35]:
gd.best_params_

{'criterion': 'gini', 'max_depth': 19, 'splitter': 'best'}

In [36]:
gd.best_score_

0.8604034929238182

In [37]:
gd.best_estimator_

DecisionTreeClassifier(max_depth=19)

In [68]:
Dt = DecisionTreeClassifier(criterion='gini',max_depth = 19, splitter = 'best')
Dt.fit(x_train,y_train)
Dt.score(x_test,y_test) 
print(Dt.score(x_test,y_test))

0.8921568627450981


In [ ]:
validation_D = cross_val_score(DecisionTreeClassifier(),x_train,y_train,cv=KFold(n_splits=10))
validation_D.mean()

0.853048780487805

In [ ]:
print(f'''DecisionTree:
    Prediction Score: {Dt.score(x_test,y_test)}
    Validation Score: {validation_D.mean()}
    Diferrence: {(validation_D.mean()-(Dt.score(x_test,y_test)))}''')

DecisionTree:
    Prediction Score: 0.8921568627450981
    Validation Score: 0.853048780487805
    Diferrence: -0.03910808225729312


In [39]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
lis = {'criterion':['gini','entropy'], 
        'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth':[i for i in range(0,20)]}

In [40]:
from sklearn.ensemble import ExtraTreesClassifier

In [41]:
fg = GridSearchCV(ExtraTreesClassifier(),param_grid= lis)
fg.fit(x_train,y_train)


GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19],
                         'max_features': ['auto', 'sqrt', 'log2']})

In [42]:
fg.best_params_

{'criterion': 'gini', 'max_depth': 16, 'max_features': 'sqrt'}

In [43]:
fg.best_score_

0.9315567600120446

In [88]:
Et =(ExtraTreesClassifier(criterion = 'gini',max_depth = 16,max_features = 'log2'))
Et.fit(x_train,y_train)
Et.score(x_test,y_test)

0.9509803921568627

In [ ]:
validation_E = cross_val_score(ExtraTreesClassifier(),x_train,y_train ,cv= KFold(n_splits=10))
validation_E

array([0.95121951, 0.92682927, 0.97560976, 0.97560976, 0.87804878,
       0.97560976, 0.97560976, 0.95121951, 1.        , 0.925     ])

In [ ]:
validation_E.mean()

0.9534756097560976

In [ ]:
print(f'''ExtraTreesClassifier:
    Prediction Score: {Et.score(x_test,y_test)}
    Validation Score: {validation_E.mean()}
    Diferrence: {(validation_E.mean()-(Et.score(x_test,y_test)))}''')

ExtraTreesClassifier:
    Prediction Score: 0.9509803921568627
    Validation Score: 0.9534756097560976
    Diferrence: 0.002495217599234878


In [47]:
from sklearn.ensemble import RandomForestClassifier

In [93]:
Rf = RandomForestClassifier()
Rf.fit(x_train,y_train)
Rf.score(x_test,y_test)

0.9215686274509803

In [ ]:

validation_R = cross_val_score(RandomForestClassifier(),x_train,y_train ,cv= KFold(n_splits=10))
validation_R

array([0.95121951, 0.95121951, 1.        , 0.97560976, 0.85365854,
       0.90243902, 0.97560976, 0.95121951, 1.        , 0.95      ])

In [ ]:
validation_R.mean()

0.9510975609756096

In [ ]:
print(f'''RandomForestClassifier:
    Prediction Score: {Rf.score(x_test,y_test)}
    Validation Score: {validation_R.mean()}
    Diferrence: {(validation_R.mean()-(Rf.score(x_test,y_test)))}''')

RandomForestClassifier:
    Prediction Score: 0.9215686274509803
    Validation Score: 0.9510975609756096
    Diferrence: 0.029528933524629264


# Support Vector Machine(SCV)

In [97]:
from sklearn.svm import SVC

In [105]:
svc_list = {'C': [0.1, 1, 10, 100, 1000], 
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
			'kernel': ['rbf']} 

svc = GridSearchCV(SVC(), param_grid= svc_list, refit = True, verbose = 3) 
 
svc.fit(x_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.512 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.500 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.500 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.506 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.506 total time=   0.1s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.622 total time=   0.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.549 total time=   0.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.561 total time=   0.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.519 total time=   0.1s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.519 total time=   0.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.512 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [106]:
svc.best_score_

0.9436013249021379

In [107]:
svc.best_params_

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}

In [103]:
sv= SVC(C=1,kernel='rbf',gamma=1)
sv.fit(x_train,y_train)
sv.score(x_test,y_test)

0.9607843137254902

In [112]:
validation_sv = cross_val_score(SVC(),x_train,y_train,cv=KFold(n_splits=10),scoring='accuracy')
validation_sv.mean()

0.7303658536585366

In [109]:
print(f'''SVC:
    Prediction Score: {sv.score(x_test,y_test)}
    Validation Score: {validation_sv.mean()}
    Diferrence: {(validation_sv.mean()-(sv.score(x_test,y_test)))}''')

SVC:
    Prediction Score: 0.9607843137254902
    Validation Score: 0.7303658536585366
    Diferrence: -0.23041846006695366


# KNN

In [110]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_list = {'n_neighbors': [3,5,7,9,11,13,15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']}
knn_ = GridSearchCV(KNeighborsClassifier(),x_train,y_train,cv= KFold(n_splits=10),scoring='accuracy')

In [51]:
# # Add after your data preprocessing
# from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# # Define cross-validation strategy
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Function for comprehensive cross-validation evaluation
# def evaluate_model_cv(model, X, y, cv_strategy=cv):
#     """
#     Perform cross-validation and return multiple metrics
#     """
#     print(f"\n{'='*60}")
#     print(f"Cross-Validation Results for {model.__class__.__name__}")
#     print('='*60)
    
#     # Calculate cross-validation scores for different metrics
#     cv_accuracy = cross_val_score(model, X, y, cv=cv_strategy, scoring='accuracy')
#     cv_precision = cross_val_score(model, X, y, cv=cv_strategy, scoring='precision')
#     cv_recall = cross_val_score(model, X, y, cv=cv_strategy, scoring='recall')
#     cv_f1 = cross_val_score(model, X, y, cv=cv_strategy, scoring='f1')
#     cv_roc_auc = cross_val_score(model, X, y, cv=cv_strategy, scoring='roc_auc')
    
#     # Print results
#     print(f"Accuracy:  {cv_accuracy.mean():.4f} (+/- {cv_accuracy.std()*2:.4f})")
#     print(f"Precision: {cv_precision.mean():.4f} (+/- {cv_precision.std()*2:.4f})")
#     print(f"Recall:    {cv_recall.mean():.4f} (+/- {cv_recall.std()*2:.4f})")
#     print(f"F1-Score:  {cv_f1.mean():.4f} (+/- {cv_f1.std()*2:.4f})")
#     print(f"ROC-AUC:   {cv_roc_auc.mean():.4f} (+/- {cv_roc_auc.std()*2:.4f})")
    
#     # Get predictions from cross-validation
#     y_pred = cross_val_predict(model, X, y, cv=cv_strategy)
    
#     # Confusion matrix
#     cm = confusion_matrix(y, y_pred)
#     print(f"\nConfusion Matrix:")
#     print(f"[[TN={cm[0,0]} FP={cm[0,1]}]")
#     print(f" [FN={cm[1,0]} TP={cm[1,1]}]]")
    
#     # Detailed classification report
#     print(f"\nClassification Report:")
#     print(classification_report(y, y_pred, target_names=['Alive', 'Dead']))
    
#     return {
#         'accuracy': cv_accuracy,
#         'precision': cv_precision,
#         'recall': cv_recall,
#         'f1': cv_f1,
#         'roc_auc': cv_roc_auc,
#         'model_name': model.__class__.__name__
#     }

# # Now let's evaluate your models with cross-validation

# # 1. Decision Tree with best parameters
# dt_best = DecisionTreeClassifier(criterion='gini', max_depth=2, splitter='random', random_state=42)
# dt_results = evaluate_model_cv(dt_best, X_final, Y)

# # 2. Extra Trees with best parameters
# et_best = ExtraTreesClassifier(
#     criterion='gini', 
#     max_depth=2, 
#     max_features='sqrt',
#     random_state=42,
#     n_estimators=100,
#     n_jobs=-1
# )
# et_results = evaluate_model_cv(et_best, X_final, Y)

# # 3. Random Forest (default)
# rf = RandomForestClassifier(random_state=42, n_jobs=-1)
# rf_results = evaluate_model_cv(rf, X_final, Y)

# # 4. Add more models for comparison
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import GradientBoostingClassifier

# # Logistic Regression
# lr = LogisticRegression(random_state=42, max_iter=1000)
# lr_results = evaluate_model_cv(lr, X_final, Y)

# # Support Vector Machine
# svm = SVC(random_state=42, probability=True)
# svm_results = evaluate_model_cv(svm, X_final, Y)

# # K-Nearest Neighbors
# knn = KNeighborsClassifier(n_neighbors=5)
# knn_results = evaluate_model_cv(knn, X_final, Y)

# # Gradient Boosting
# gb = GradientBoostingClassifier(random_state=42)
# gb_results = evaluate_model_cv(gb, X_final, Y)

# # Visualize cross-validation results
# import matplotlib.pyplot as plt
# import numpy as np

# def plot_cv_results(results_list):
#     """
#     Plot cross-validation results comparison
#     """
#     fig, axes = plt.subplots(2, 3, figsize=(15, 10))
#     axes = axes.flatten()
    
#     metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
#     titles = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
    
#     for idx, (metric, title) in enumerate(zip(metrics, titles)):
#         ax = axes[idx]
        
#         # Create data for boxplot
#         data = []
#         model_names = []
        
#         for result in results_list:
#             if metric in result:
#                 data.append(result[metric])
#                 model_names.append(result['model_name'])
        
#         # Create boxplot
#         bp = ax.boxplot(data, patch_artist=True)
        
#         # Color the boxes
#         colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink', 'lightgray', 'orange']
#         for patch, color in zip(bp['boxes'], colors):
#             patch.set_facecolor(color)
        
#         ax.set_title(f'{title} Comparison')
#         ax.set_xticklabels(model_names, rotation=45, ha='right')
#         ax.set_ylabel('Score')
#         ax.grid(True, alpha=0.3)
    
#     # Add overall comparison plot
#     ax = axes[5]
#     model_names = []
#     mean_scores = []
    
#     for result in results_list:
#         model_names.append(result['model_name'])
#         # Calculate mean of all metrics
#         mean_score = np.mean([
#             result['accuracy'].mean(),
#             result['precision'].mean(),
#             result['recall'].mean(),
#             result['f1'].mean(),
#             result['roc_auc'].mean()
#         ])
#         mean_scores.append(mean_score)
    
#     bars = ax.barh(model_names, mean_scores, color='skyblue')
#     ax.set_xlabel('Average Score')
#     ax.set_title('Overall Model Performance')
#     ax.grid(True, alpha=0.3)
    
#     # Add value labels on bars
#     for bar in bars:
#         width = bar.get_width()
#         ax.text(width + 0.01, bar.get_y() + bar.get_height()/2,
#                 f'{width:.3f}', ha='left', va='center')
    
#     plt.tight_layout()
#     plt.show()

# # Plot all results
# all_results = [dt_results, et_results, rf_results, lr_results, svm_results, knn_results, gb_results]
# plot_cv_results(all_results)

# # Function for nested cross-validation (for hyperparameter tuning)
# def nested_cv_hyperparameter_tuning(model, param_grid, X, y):
#     """
#     Perform nested cross-validation for hyperparameter tuning
#     """
#     from sklearn.model_selection import GridSearchCV
    
#     # Outer CV for evaluation
#     outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     # Store results
#     outer_scores = []
#     best_params_list = []
    
#     print(f"\n{'='*60}")
#     print(f"Nested CV for {model.__class__.__name__}")
#     print('='*60)
    
#     for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X, y), 1):
#         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
#         # Inner CV for hyperparameter tuning
#         inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        
#         # Grid search in inner loop
#         grid_search = GridSearchCV(
#             estimator=model,
#             param_grid=param_grid,
#             cv=inner_cv,
#             scoring='accuracy',
#             n_jobs=-1,
#             verbose=0
#         )
        
#         grid_search.fit(X_train, y_train)
        
#         # Best model from inner CV
#         best_model = grid_search.best_estimator_
        
#         # Evaluate on outer test fold
#         score = best_model.score(X_test, y_test)
#         outer_scores.append(score)
#         best_params_list.append(grid_search.best_params_)
        
#         print(f"Fold {fold_idx}: Best params = {grid_search.best_params_}, Score = {score:.4f}")
    
#     print(f"\nMean CV Score: {np.mean(outer_scores):.4f} (+/- {np.std(outer_scores)*2:.4f})")
#     print(f"Best parameters across folds:")
#     for i, params in enumerate(best_params_list, 1):
#         print(f"  Fold {i}: {params}")
    
#     return outer_scores, best_params_list

# # Example: Nested CV for Decision Tree
# dt_param_grid = {
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# dt_scores, dt_best_params = nested_cv_hyperparameter_tuning(
#     DecisionTreeClassifier(random_state=42),
#     dt_param_grid,
#     X_final,
#     Y
# )

# # Final model training with best parameters from nested CV
# print(f"\n{'='*60}")
# print("Training Final Model with Best Parameters")
# print('='*60)

# # Find most frequent best parameters
# from collections import Counter
# param_counts = Counter(str(p) for p in dt_best_params)
# most_common_param = eval(param_counts.most_common(1)[0][0])

# final_model = DecisionTreeClassifier(**most_common_param, random_state=42)
# final_model.fit(X_final, Y)

# print(f"Final model parameters: {most_common_param}")

# # Feature importance analysis
# if hasattr(final_model, 'feature_importances_'):
#     feature_importance = pd.DataFrame({
#         'feature': X_final.columns,
#         'importance': final_model.feature_importances_
#     }).sort_values('importance', ascending=False)
    
#     print(f"\nTop 10 Most Important Features:")
#     print(feature_importance.head(10))
    
#     # Plot feature importance
#     plt.figure(figsize=(10, 6))
#     plt.barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
#     plt.xlabel('Importance Score')
#     plt.title('Top 10 Feature Importance')
#     plt.gca().invert_yaxis()
#     plt.tight_layout()
#     plt.show()